Part 1 contains information about:
1. Merging of the tables
2. Data quality checks
3. EDA -- pattern in regard with the target variable

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
#  load the two datasets
campaign_df = pd.read_csv("../Datasets/Campaign.csv", sep=",", header=0)
mortgage_df = pd.read_csv("../Datasets/Mortgage.csv", sep=",", header=0)

campaign_df.head(3)

,participant_id,name_title,first_name,last_name,age,postcode,marital_status,education,job_title,occupation_level,education_num,familiarity_FB,view_FB,interested_insurance,company_email,created_account
0,8.997000e+12,Mr.,Dale,Coles,39,EH4 9FL,Never-married,Bachelors,Furniture conservator/restorer,1,17,7,9,0,jrhodes@jones.com,No
1,2.764970e+12,NaN,Joel,Allen,50,LS5H 0JG,Married-civ-spouse,Bachelors,"Administrator, sports",4,17,9,6,1,william32@carey.com,No
2,8.454880e+12,Mr.,Craig,Davis,38,EH8W 2QJ,Divorced,HS-grad,Armed forces training and education officer,12,12,5,4,1,smithvanessa@coles.org,No


In [4]:
# check for duplicate values and NAs
def simple_checks(df):
    """Run sumple checks on the dataset
    """
    n_duplicates = df.shape[0] - df.drop_duplicates().shape[0]
    print(f"Campaign data contain {n_duplicates} duplicate values.")

    for col in df.columns:
        n_nulls = df[col].isnull().sum()
        n_distinct = df[col].nunique()
        print(f"Value {col} contains {n_nulls} null values and {n_distinct} distinct values.")

simple_checks(campaign_df)
print("--- --- --- --- ---")
simple_checks(mortgage_df)

Campaign data contain 0 duplicate values.
Value participant_id contains 0 null values and 31639 distinct values.
Value name_title contains 12201 null values and 5 distinct values.
Value first_name contains 0 null values and 182 distinct values.
Value last_name contains 0 null values and 500 distinct values.
Value age contains 0 null values and 73 distinct values.
Value postcode contains 0 null values and 31398 distinct values.
Value marital_status contains 0 null values and 7 distinct values.
Value education contains 0 null values and 16 distinct values.
Value job_title contains 0 null values and 691 distinct values.
Value occupation_level contains 0 null values and 20 distinct values.
Value education_num contains 0 null values and 16 distinct values.
Value familiarity_FB contains 0 null values and 10 distinct values.
Value view_FB contains 0 null values and 10 distinct values.
Value interested_insurance contains 0 null values and 2 distinct values.
Value company_email contains 0 null 

Join the data

In [7]:
Campaign_copy_df = campaign_df.copy()

In [8]:
# approach 1 -- full name
full_names = Campaign_copy_df["name_title"].fillna("") + " " + Campaign_copy_df["first_name"] + " " + Campaign_copy_df["last_name"]
full_names = [name.strip() for name in full_names]
Campaign_copy_df = Campaign_copy_df.assign(full_name=full_names)

n_unique_names = Campaign_copy_df["full_name"].nunique()
print(f"The campaign data contains {n_unique_names} unique full names. That accounts for {round(n_unique_names/Campaign_copy_df.shape[0]*100, 2)}% of all entries in the campaign dataset")

Campaign_copy_df\
    .groupby(["full_name"])\
    ["full_name"].count()\
    .rename("name_count")\
    .reset_index()\
    .sort_values("name_count")\
    .tail(4)

The campaign data contains 28942 unique full names. That accounts for 90.27% of all entries in the campaign dataset


,full_name,name_count
27515,Russell Smith,5
27332,Ronald Williams,6
18381,Mr. Mohammad Jones,6
7446,Lawrence Jones,7


In [11]:
# approach 2 -- full name + age
names_w_age = list(Campaign_copy_df["full_name"] + " - " + Campaign_copy_df["age"].astype(str))
Campaign_copy_df['names_w_age'] = names_w_age

Campaign_copy_df\
    .groupby(["names_w_age"])\
    ["participant_id"].count()\
    .reset_index()\
    .sort_values("participant_id")\
    .tail(4)

,names_w_age,participant_id
15878,Mr. Donald Ward - 45,2
12664,Mohammed Davies - 32,2
2295,Colin Smith - 41,2
17173,Mr. Graeme Smith - 21,3


In [14]:
# approach 3 -- full name + age + location
# since name and age are not enough to define a unique customer identifier for the people that were targeted by the previous campaign
# we need to add information about the city each person recides
# this can be done with the aid of two python packages, namely pgeocode and geopy
# I will use the first one to find the coordinates of the location (pgeocode does not return a city name and accept a 3letter zip code)
# and the secord one (geopy) to consume the coordinates of the place and return a city name
import pgeocode
from geopy.geocoders import Nominatim

# initialize the Nominatim API
geolocator = Nominatim(user_agent="campaign_eda")
# set the country for pgeovode to Great Britain
nomi = pgeocode.Nominatim("GB")


# --- --- --- --- helper --- --- --- ---
def postcode_to_city_converter(post_code: str) -> str:
    """Takes as input any postal code in string format and returns the city name
    """
    # keep the first three letters of the postal code
    if len(post_code)>=3:
        three_letter_postcode = post_code[:3]
    else:
        # len(pos_code)<3
        raise ValueError("Postal Code needs to be at least 3 characters long.")

    # get the coordinates of the location of the postal code
    lat, lon = nomi.query_postal_code(three_letter_postcode)[["latitude", "longitude"]]
    coordinates = str(lat) + "," + str(lon)

    # extract the geopy geolocation
    geolocation = geolocator.reverse(coordinates)
    address = geolocation.raw["address"]

    # if city in address.keys(): address["city"]
    # if town in address.keys(): address["town"]
    # return the city name
    return address

# The following does not return Edinburg -- no city
postcode_to_city_converter("EH4 9FL")

{'house_number': '288',
 'road': 'Craigcrook Road',
 'neighbourhood': 'Craigcrook',
 'suburb': 'Blackhall',
 'county': 'City of Edinburgh',
 'state': 'Alba / Scotland',
 'postcode': 'EH4 7BA',
 'country': 'United Kingdom',
 'country_code': 'gb'}

In [15]:
# approach 4 -- row id as customer id
# since there are a few issues with the city locator from the geolocator python packages
# we can now assume that the two dataset have the row id as customer id and join them on that information
# this has been validates by viewing only, a variety of results
full_df = pd.concat([campaign_df, mortgage_df], axis=1, join="inner")